In [88]:
import numpy as np
import sympy
from sympy import symbols, factorial, Function, summation, binomial, product

n, i, j, m = symbols('n, i, j, m', integer=True, real=True)

t = symbols('t')
P = Function('P')

B = summation(binomial(n, i)*(1 - t)**(n-i)*t**i*P(i), (i, 0, n))

In [89]:
def bernstein(i, n, t):
    "bernstein basis polynomial"
    return binomial(n, i)*(1 - t)**(n-i)*t**i

def bezier(i, n, t, P):
    return summation(bernstein(i, n, t)*P(i), (i, 0, n))


bezier(i, n, t, P)

Sum(t**i*(1 - t)**(-i + n)*P(i)*binomial(n, i), (i, 0, n))

In [90]:
n*summation(bernstein(i, n-1, t)*(P(i+1) - P(i)), (i, 0, n-1))

n*Sum(t**i*(1 - t)**(-i + n - 1)*(-P(i) + P(i + 1))*binomial(n - 1, i), (i, 0, n - 1))

In [91]:
class Bezier:
    
    def __init__(self, P : list[sympy.Matrix]):
        self.P = P
        self.n = len(P)
    
    def eval(self, t):
        P = self.P
        n = self.n
        B = None
        for i in range(self.n):
            Bi = bernstein(i, n, t)*P[i]
            if B is None:
                B = Bi
            else:
                B += Bi        
        return B

    def derivative(self):
        deltaP = [
            self.P[i] - self.P[i-1]
            for i in range(1, self.n)]
        return Bezier(P=deltaP)
    
    def __repr__(self):
        return repr(self.P)

In [92]:
P = [ sympy.MatrixSymbol(f'P_{i}', 2, 1) for i in range(3) ]
pos = Bezier(P)
vel = pos.derivative()
acc = vel.derivative()
P

[P_0, P_1, P_2]

In [93]:
acc.eval(t).subs({P[0], sympy.Matrix([1, 2])})

TypeError: unhashable type: 'MutableDenseMatrix'

In [32]:
P[:, 0] - P[:, 1]

P[:, :1] - P[:, 1:2]

In [20]:
T = 1
T_0 = 0
beta = (t - T_0)/T

pos.eval(beta).applyfunc(lambda x: x.simplify())

Matrix([
[-7*t**3 + 3*t**2 + 3*t + 1],
[-7*t**3 + 3*t**2 + 3*t + 1]])

In [23]:
C = sympy.factorial(n)/sympy.factorial(n - j)*summation((-1)**(i + j)*P(i)/(factorial(i)*factorial(j - i)), (i, 0, j))
n0 = 6

P_vect = sympy.Matrix([P(i) for i in range(n0)])

C_matrix = sympy.Matrix([C.subs({j: j0, n: n0}).doit() for j0 in range(n0)])
C_matrix

A = C_matrix.jacobian(P_vect)

C_to_B = np.array(A.inv(), dtype=float)
C_to_B

TypeError: 'MutableDenseMatrix' object is not callable

In [7]:
import casadi as ca
import numpy as np

In [65]:
def de_casteljau(t, coefs):
    # https://en.wikipedia.org/wiki/De_Casteljau%27s_algorithm
    n = coefs.shape[1]  # number of points
    beta = ca.SX(coefs)
    for j in range(1, n):
        for k in range(n - j):
            beta[:, k] = (1 - t) * beta[:, k] + t * beta[:, k + 1]
    return beta[:, 0]

def bezier_derivative(coefs):
    n = coefs.shape[0]
    coeffs_deriv = ca.SX(n, coefs.shape[1] - 1)
    for i in range(n-1):
        coeffs_deriv[:, i] = coefs[:, i + 1] - coefs[:, i]
    return coeffs_deriv


t = ca.SX.sym('t')
P = ca.SX.sym('P', 1, 6)  # bezier position points
P_dot = bezier_derivative(P) # bezier velocity points
P_ddot = bezier_derivative(P_dot) # bezier accel points

f_pos = ca.Function('f_pos', [t, P], [de_casteljau(t, P)], ['t', 'P'], ['B'])
f_vel = ca.Function('f_vel', [t, P], [de_casteljau(t, P_dot)], ['t', 'P_dot'], ['B'])
f_acc = ca.Function('f_acc', [t, P], [de_casteljau(t, P_ddot)], ['t', 'P_ddot'], ['B'])


In [86]:
P0 = np.array([
    [1, 2, 5, 6]
])


f_pos(1, P0)

RuntimeError: .../casadi/core/function_internal.hpp:1257: Input 1 (P) has mismatching shape. Got 1-by-4. Allowed dimensions, in general, are:
 - The input dimension N-by-M (here 1-by-6)
 - A scalar, i.e. 1-by-1
 - M-by-N if N=1 or M=1 (i.e. a transposed vector)
 - N-by-M1 if K*M1=M for some K (argument repeated horizontally)
 - N-by-P*M, indicating evaluation with multiple arguments (P must be a multiple of 1 for consistency with previous inputs)

In [87]:
import matplotlib.pyplot as plt
t = np.linspace(0, 1)
f_pos_eval = np.hstack([ f_pos(ti, P0) for ti in t ]).T
f_vel_eval = np.hstack([ f_vel(ti, P0) for ti in t ]).T
f_acc_eval = np.hstack([ f_acc(ti, P0) for ti in t ]).T

plt.plot(t, f_pos_eval, label='pos')
plt.plot(t, f_vel_eval, label='vel')
plt.plot(t, f_acc_eval, label='acc')
plt.legend()

RuntimeError: .../casadi/core/function_internal.hpp:1257: Input 1 (P) has mismatching shape. Got 1-by-4. Allowed dimensions, in general, are:
 - The input dimension N-by-M (here 1-by-6)
 - A scalar, i.e. 1-by-1
 - M-by-N if N=1 or M=1 (i.e. a transposed vector)
 - N-by-M1 if K*M1=M for some K (argument repeated horizontally)
 - N-by-P*M, indicating evaluation with multiple arguments (P must be a multiple of 1 for consistency with previous inputs)